#Analysis of physiological responses to emotional visual stimuli

This notebook demonstrates the analysis of physiological recordings obtained while the subject observed emotional visual stimuli.

To get started, clone the course github containing our experimental files

In [ ]:
!git clone https://github.com/StolkArjen/human-interaction.git
# to remove the folder, use: !rm -rf human-interaction
# to clear all outputs, go to Edit > Clear all outputs, followed by Runtime > Restart

Add the code folder to the path

In [ ]:
import os, sys
sys.path.append(os.getcwd() + '/human-interaction/code')

Read data from a subject into the workspace



In [ ]:
from read_iworx import read_iworx

data, event = read_iworx(os.getcwd() + '/human-interaction/data/EMG_WI22/BD')

print(event.value)
print(event.sample)
print(len(data.trial))

Data has the following nested fields:
    .trial
    .time
    .label

Event has the following nested fields:
    .type
    .sample
    .value

Let's sort our stimuli per emotion type, either positive (+) or negative (-), and get their sample indices

In [ ]:
posidx = [i for i, e in enumerate(event.value) if e[0] == '+']
negidx = [i for i, xxxxx e[0] == '-']

possmp = [event.sample[p] for p in posidx]
negsmp = [event.sample[p] for p in negidx]

print(possmp)
print(negsmp)

Got an error? My cat may have stepped on the keyboard. Can you fix it?

Okay, let's continue once fixed and extract data samples from around the presentation of each stimulus based on the above sample indices

In [ ]:
import numpy as np

pre = 1000 # samples corresponding to msec
post = 1000

posdat = []
for t in range(len(data.time)): # loop through trials
  for e in range(len(posidx)): # loop through events
      if possmp[e] > data.time[t][0] and possmp[e] < data.time[t][-1]:
          idx = data.time[t].index(possmp[e])
          posdat.append(data.trial[t][:,idx-pre:idx+post])

negdat = []

Any code missing? Hint: negatives?

Now that we have the responses to stimuli, let's plot them together

In [ ]:
import matplotlib.pyplot as plt

time = range(-pre, post, 1)
for t in range(len(posdat)):
  fig, axs = plt.subplots(1,5, figsize=(15,5))
  for p in range(len(posdat[t])):
    axs[p].plot(time, posdat[t][p,:], 'b')
    axs[p].plot(time, negdat[t][p,:], 'r')
    axs[p].title.set_text(data.label[p])

Let's take a look at the averages

In [ ]:
posavg = np.nanmean(posdat, axis=0)
negavg = np.nanmean(negdat, axis=0)

fig, axs = plt.subplots(1,5, figsize=(15,5))
for p in range(len(posavg)):
  axs[p].plot(time, posavg[p,:], 'b')
  axs[p].plot(time, negavg[p,:], 'r')
  axs[p].title.set_text(data.label[p])

Let's plot them as changes from baseline, i.e. subtracting the time before the stimulus

In [ ]:
poschange = []
for t in range(len(posdat)):
  baseavg = np.mean(posdat[t][:,:pre],axis=1)
  baseline = np.tile(baseavg[:,np.newaxis],(1,pre+post))
  poschange.append(posdat[t]-baseline)
poschangeavg = np.nanmean(poschange, axis=0)

negchange = []
for t in range(len(negdat)):
  baseavg = np.mean(negdat[t][:,:pre],axis=1)
  baseline = np.tile(baseavg[:,np.newaxis],(1,pre+post))
  negchange.append(negdat[t]-baseline)
negchangeavg = np.nanmean(negchange, axis=0)

fig, axs = plt.subplots(1,5, figsize=(15,5))
for p in range(len(poschangeavg)):
  axs[p].plot(time, poschangeavg[p,:], 'b')
  axs[p].plot(time, negchangeavg[p,:], 'r')
  axs[p].title.set_text(data.label[p])

What shall we investigate next? What about the relationship between heart rate and skin conductance? 

To do this, we first extract data from the first stimulus until the last

In [ ]:
for i, t in enumerate(data.time):
  if event.sample[0] > data.time[i][0] and event.sample[-1] < data.time[i][-1]: # if events fall within this trial
        begin = data.time[i].index(event.sample[0])
        end = data.time[i].index(event.sample[-1])
        dat = data.trial[i][:,begin:end] 

# check for matching durations (if this breaks, it might be due to events having been recorded by separate trials)
print(event.sample[-1]-event.sample[0]) # in seconds
print(len(dat[0])) # in samples, at 1000 Hz resolution

Next, we correlate the heart rate and skin conductance channels

In [ ]:
hridx = data.label.index('Heart Rate')
scidx = data.label.index('Skin Conductance')

fig = plt.figure()
plt.plot(dat[hridx], 'g')
plt.plot(dat[scidx], 'y')

R = np.corrcoef(dat[hridx], dat[scidx])[1][0]
print(R)

For a thought experiment, or perhaps actual if time allows, how would you go about correlating data from two or more subjects? 